# Evaluate LG3 , LG4 (LG3 + AF filter) : for jsuthusky  ,  KI pathogenic

1. LG4: if use AF filter , then add function `def filter_AF`; 
2. LG3: if not, only see the LG3 pred, then comment the function 

Method:  
* check the number of predictions for benign and pathogenic



In [1]:
import gzip
import pandas as pd
import numpy as np
from joblib import dump, load
from sklearn.compose import make_column_transformer
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from category_encoders import BinaryEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

/Users/nancy/miniconda3/envs/bioinfo/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# predic justhusky

In [2]:
""" FIRST of ALL-- user need"""

##### please set files
# in
# vcfgz_in_file = '/Users/nancy/Desktop/RS_projects/data/03_ML/03_predict_new/01_featureV1/justhusky_gatkcomb_rhocall_norm_af_mt_frqf_cadd_vep_parsed_ranked.vcf.gz'
variant_consequences_order = './variant_consequences.txt'

# out
# pred_out_simple = '/Users/nancy/Desktop/RS_projects/result/04_ML_modify/02_evaluate/01_Clinvar_pred-29feats_CJP5-5_LG.csv'


##### load fitted preprocessor and model
preprocessor = load('/Users/nancy/Desktop/RS_projects/result/04_ML_modify/01_retrainModel_ClinvarJusthuskyNewpatho/models/02_12feats_ClinJustPatho5-5_preprocessor.joblib') # data preprocessor
LG_model = load('/Users/nancy/Desktop/RS_projects/result/04_ML_modify/01_retrainModel_ClinvarJusthuskyNewpatho/models/02_12feats_ClinJustPatho5-5_LG.joblib')         # logistic regression model


In [3]:
## predict


df=pd.read_csv('/Users/nancy/Desktop/RS_projects/result/03_ML/03_predict_new/01_featureV1/01_justhusky_features_noclnsig.csv',sep=';')
df['idx'] = df.index  # add index column

""" 2.1 choose featuresV1"""

featureV1 = ['idx','CADD', 'AF_TGP', 'Frq', 'IMPACT','GNOMADAF_popmax', 'Hom', 'ORIGIN',
       'SPIDEX', 'SWEGENAF', 'Consequence', 'BIOTYPE', 'SIFT',
       'PolyPhen', 'MES-SWA_acceptor_alt', 'MES-SWA_acceptor_diff',
       'MES-SWA_donor_alt', 'MES-SWA_donor_diff', 'MaxEntScan_alt',
       'MaxEntScan_diff', 'GERP++_RS', 'REVEL_score',
       'phastCons100way_vertebrate', 'phyloP100way_vertebrate', 'LoFtool',
       'pLI_gene_value', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL',
       'SpliceAI_pred_DS_DG', 'SpliceAI_pred_DS_DL']         # remove CLNSIG,   add index: 29+1 total

df_1 = df[featureV1]



""" 2.2 preprocessing and predict """

## predict function :
def predict_score(X,model,df_ori):
    # Use preprocessor and model to predict on new data
    X_processed = preprocessor.transform(X)
    pred = model.predict(X_processed)
    prob = model.predict_proba(X_processed)

    # put prediction and score back to dataframe with same index
    df_out = pd.DataFrame(data = [pred,prob[:,1]]).T
    df_out.columns = ['prediction','score']
    df_out['score']=df_out['score'].round(3)
    df_out = pd.concat([X['idx'],df_out],axis=1)

    #  write model outputs in original df at same index rows
    df_copy = df_ori.copy() 
    df_merged = pd.merge(df_ori,df_out, on='idx')    # get only rows merged when index is same
    df_copy.loc[df_copy['idx'].isin(df_merged['idx']),['prediction','score']] = df_merged[['prediction','score']].values  # put cells of merged in original dataframe
    lst = ['idx','prediction','score']
    return df_copy[lst]

# data preprocessing and predicted by models
df_lg = predict_score(df_1,LG_model,df)

# df_lg.to_csv(pred_out_simple,index=False)




/var/folders/q0/2p8kfxsn7s73349btf1_g07h0000gp/T/ipykernel_95922/2892436993.py:4: DtypeWarning: Columns (22,23,24,25,26,27,31,37,39,40,41,48,49,50,51,53,55,57,66,90,92,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/Users/nancy/Desktop/RS_projects/result/03_ML/03_predict_new/01_featureV1/01_justhusky_features_noclnsig.csv',sep=';')


In [4]:
""" function: filter prediction by AF > 0.01:
if any AF > 0.01 & patho, change patho to benign 

output: precdiction by model after filtering """


# function:  to get the maximum non-NaN value in a row
def min_nonnan(row):
    nonnan_vals = [val for val in row if not np.isnan(val)]
    return max(nonnan_vals) if nonnan_vals else np.nan


## function: transfer `predicted patho`` with any AF>=0.01, to `benign`
def filter_AF(df_extracted_ori,df_pred_out):

    af = ['AF_ESP', 'AF_EXAC', 'AF_TGP', 'Frq', 'GNOMADAF_popmax', 'SWEGENAF']
    df_af = df_extracted_ori[af]  # Extract all related to AF parameters from the data

    df_merge_af = pd.concat([df_pred_out, df_af], axis=1)
    df_merge_af['AF_min'] = df_merge_af[af].apply(min_nonnan, axis=1)   # create a new 'AF_min' column

    
    df_filter = df_merge_af.copy()
    df_filter.loc[(df_filter['prediction'] == 1) & (df_filter['AF_min'] >= 0.01), ['prediction']] = 0   # Change 'patho' to 'benign' when min 'AF' is >= 0.01

    # Extract the 'prediction' and 'score' columns from the filtered data
    df_pred_out_1 = df_filter[['prediction', 'score']]

    # Return the value counts of 'prediction'
    return df_pred_out_1

df_lg = filter_AF(df,df_lg)

## evaluate   Justhusy - prediction results: for all Justhusky variants

In [5]:
df_lg.prediction.value_counts()

0.0    4575006
1.0        268
Name: prediction, dtype: int64

# predic KI pathogenic

In [6]:
## predict


df=pd.read_csv('/Users/nancy/Desktop/RS_projects/result/03_ML/03_predict_new/02_featureV2/015_NEWpatho_features_noclnsig.csv',sep=';')
df['idx'] = df.index  # add index column

""" 2.1 choose featuresV1"""

featureV1 = ['idx','CADD', 'AF_TGP', 'Frq', 'IMPACT','GNOMADAF_popmax', 'Hom', 'ORIGIN',
       'SPIDEX', 'SWEGENAF', 'Consequence', 'BIOTYPE', 'SIFT',
       'PolyPhen', 'MES-SWA_acceptor_alt', 'MES-SWA_acceptor_diff',
       'MES-SWA_donor_alt', 'MES-SWA_donor_diff', 'MaxEntScan_alt',
       'MaxEntScan_diff', 'GERP++_RS', 'REVEL_score',
       'phastCons100way_vertebrate', 'phyloP100way_vertebrate', 'LoFtool',
       'pLI_gene_value', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL',
       'SpliceAI_pred_DS_DG', 'SpliceAI_pred_DS_DL']         # remove CLNSIG,   add index: 29+1 total

df_1 = df[featureV1]



""" 2.2 preprocessing and predict """

## predict function :
def predict_score(X,model,df_ori):
    # Use preprocessor and model to predict on new data
    X_processed = preprocessor.transform(X)
    pred = model.predict(X_processed)
    prob = model.predict_proba(X_processed)

    # put prediction and score back to dataframe with same index
    df_out = pd.DataFrame(data = [pred,prob[:,1]]).T
    df_out.columns = ['prediction','score']
    df_out['score']=df_out['score'].round(3)
    df_out = pd.concat([X['idx'],df_out],axis=1)

    #  write model outputs in original df at same index rows
    df_copy = df_ori.copy() 
    df_merged = pd.merge(df_ori,df_out, on='idx')    # get only rows merged when index is same
    df_copy.loc[df_copy['idx'].isin(df_merged['idx']),['prediction','score']] = df_merged[['prediction','score']].values  # put cells of merged in original dataframe
    lst = ['idx','prediction','score']
    return df_copy[lst]

# data preprocessing and predicted by models
df_lg_newpatho = predict_score(df_1,LG_model,df)

# df_lg.to_csv(pred_out_simple,index=False)

In [2]:
df_lg_newpatho = filter_AF(df,df_lg_newpatho)

NameError: name 'filter_AF' is not defined

### evaluate all new patho

In [8]:
df_lg_newpatho.prediction.value_counts()

1.0    789
0.0    188
Name: prediction, dtype: int64